## Carregando os dados

In [ ]:
import pandas as pd

In [ ]:
# Carregando os arquivos csv 
# part1 entre 2020-02-25 - 2020-07-31
part1 = pd.read_csv('HIST_PAINEL_COVIDBR_2020_Parte1_05nov2021.csv', sep=';')
# part2 entre 2020-08-01 - 2020-12-31
part2 = pd.read_csv('HIST_PAINEL_COVIDBR_2020_Parte2_05nov2021.csv', sep=';')
# part3 entre 2021-01-01 - 2021-06-30
part3 = pd.read_csv('HIST_PAINEL_COVIDBR_2021_Parte1_05nov2021.csv', sep=';')
# part4 entre 2021-07-01 - 2021-11-05
part4 = pd.read_csv('HIST_PAINEL_COVIDBR_2021_Parte2_05nov2021.csv', sep=';')

In [ ]:
# Juntando todas as parts em um dataframe unico
df = pd.merge(part1, part2, how='outer')
df = pd.merge(df, part3, how='outer')
df = pd.merge(df, part4, how='outer')
df

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3310454,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-11-01,44,3015268.0,515134,198,10886,9,NaN,NaN,1.0
3310455,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-11-02,44,3015268.0,515211,77,10894,8,NaN,NaN,1.0
3310456,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-11-03,44,3015268.0,515399,188,10902,8,NaN,NaN,1.0
3310457,Centro-Oeste,DF,Brasília,53,530010.0,53001.0,DISTRITO FEDERAL,2021-11-04,44,3015268.0,515497,98,10906,4,NaN,NaN,1.0


## Limpezado dos dados

In [ ]:
# Selecionado apenas os dados referentes ao Centro-Oeste.
df = df.loc[df['regiao']=='Centro-Oeste']

In [ ]:
# Os 'municipio' que possuem NaN, possue ja a soma total de 'casosNovos', 'obitosNovos' e outros
# dessa forma nao eh preciso fazer a soma de todos os 'municipio'.
df = df[df['municipio'].isnull()]

In [ ]:
# De todas as colunas as necessarias sao 'data', 'casosNovos' e 'obitosNovos'.
df = df[['data','casosNovos','obitosNovos']]

In [ ]:
# Fazendo a soma em relacao a 'data'. Dessa forma obtemos o total de 'casosNovos' e 'obitosNovos' por dia.
df = df.groupby(['data']).sum()

In [ ]:
# Convertendo o index 'data' para coluna
df = df.reset_index()

In [ ]:
# Excluindo os registros do mes de novembro de 2021
df = df.drop(df.index[[615, 616, 617, 618, 619]])

In [ ]:
# Alterando o nome das colunas 'casosNovos' e 'obitosNovos' para 'casos' e 'obitos'
df.rename(columns={'casosNovos': 'casos'}, inplace = True)
df.rename(columns={'obitosNovos': 'obitos'}, inplace = True)

In [ ]:
# Dataframe depois da limpeza e alteracoes
# data -> registros de 2020-02-25	ate 2021-10-31 de casos e obitos diarios de COVID-19 no Centro-Oeste do Brasil
# casos -> total de casos novos diarios de COVID-19 no Centro-Oeste do Brasil
# obitos -> total de obitos novos diarios de COVID-19 no Centro-Oeste do Brasil
df

,data,casos,obitos
0,2020-02-25,0,0
1,2020-02-26,0,0
2,2020-02-27,0,0
3,2020-02-28,0,0
4,2020-02-29,0,0
...,...,...,...
610,2021-10-27,2340,23
611,2021-10-28,2776,47
612,2021-10-29,2117,40
613,2021-10-30,920,16


## Separacao do conjunto de dados (casos e obitos)

In [ ]:
dfCasos = df[["data", "casos"]]
dfCasos

In [ ]:
dfCasos = df[["data", "obitos"]]
dfCasos

## Visualização dos dados

In [ ]:
import plotly.express as px
import plotly.offline as pyoff
import plotly.graph_objects as go

In [ ]:
# Plotando casos
fig = px.line(df, x="data", y="casos", title='Casos diários de COVID-19 no Centro-Oeste entre 25/02/2020 - 31/10/2021')
fig.show()

In [ ]:
# Plotando obitos
fig = px.line(df, x="data", y="obitos", title='Óbitos diários por COVID-19 no Centro-Oeste entre 25/02/2020 - 31/10/2021')
fig.update_traces(line_color='#ed3f00')
py.iplot(fig, filename='Obitos diários')

In [ ]:
# Plotando casos e obitos
import chart_studio.plotly as py
import plotly.graph_objects as go
aux = pd.pivot_table(df, values = ['casos', 'obitos'], index = 'data')

casos = go.Scatter(
    x = aux.index,
    y = aux.casos,
    mode = 'lines',
    name = 'Casos'
)
obitos = go.Scatter(
    x = aux.index,
    y = aux.obitos,
    mode = 'lines',
    name = 'Óbitos'
)

data = [casos, obitos]
layout = go.Layout(title = 'Casos e óbitos diários por COVID-19 no Centro-Oeste entre 25/02/2020 e 31/10/2021')
figure = go.Figure(data = data, layout = layout)
py.iplot(data, filename='Casos e obitos diários')

In [ ]:
from sklearn.model_selection import train_test_split
train_visu, test_visu = train_test_split(df, test_size=0.3, shuffle=False)

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=train_visu.data, y=train_visu.casos, line={'color': 'royalblue'}, name='treino')
fig.add_scattergl(x=test_visu.data, y=test_visu.casos, line={'color': 'orangered'}, name='teste')
fig.update_layout(
    title="Divisão do dataset de casos diários em 70% treino e 30% teste",
    xaxis_title="data",
    yaxis_title="casos diários",
    font=dict(
        size=12,
    )
)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=train_visu.data, y=train_visu.obitos, line={'color': 'royalblue'}, name='treino')
fig.add_scattergl(x=test_visu.data, y=test_visu.obitos, line={'color': 'orangered'}, name='teste')
fig.update_layout(
    title="Divisão do dataset de óbitos diários em 70% treino e 30% teste",
    xaxis_title="data",
    yaxis_title="óbitos diários",
    font=dict(
        size=12,
    )
)
fig.show()

# Implementação dos modelos

#### Imports

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import LSTM, GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
from datetime import datetime
import numpy
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
import math

#### Função de Separação do DataSet

In [ ]:
# Cria o DataSet
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

## LSTM: Casos

In [ ]:
# Semente do aleatório fixada
numpy.random.seed(7)

In [ ]:
# Seta os dados
dataset = df['casos'].values
dataset = dataset.astype('float32')

In [ ]:
# Normaliza para facilitar nos calculos,  entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))

In [ ]:
# Separação entre Treino e Teste
train_fraction = 0.70
train_size = int(len(dataset) * train_fraction)
test_size = len(dataset) - train_size
Treino, Teste = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# Reshape para X=t e Y=t+1
look_back = 20
TreinoX, TreinoY = create_dataset(Treino, look_back)
testX, testY = create_dataset(Teste, look_back)
TreinoX = numpy.reshape(TreinoX, (TreinoX.shape[0], TreinoX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(units=16, input_shape=(look_back, 1), return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(units=1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
start_time = datetime.now()
history = model.fit(TreinoX, TreinoY, epochs = 300, batch_size = 10)
final_time = datetime.now()

In [ ]:
aux = final_time - start_time
print(aux.total_seconds()/60)

In [ ]:
y_ajustado = model.predict(TreinoX)
aux1 = TreinoY
aux2 = y_ajustado[:,0]
y_predito = model.predict(testX)
aux3 = testY
aux4 = y_predito[:,0]

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=train_visu.data, y=aux1, line={'color': 'royalblue'}, name='treino')
fig.add_scattergl(x=train_visu.data, y=aux2, line={'color': 'green'}, name='ajuste_treino')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=test_visu.data, y=aux3, line={'color': 'royalblue'}, name='teste')
fig.add_scattergl(x=test_visu.data, y=aux4, line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
# Predizer os dados
trainPredict = model.predict(TreinoX)
testPredict = model.predict(testX)

In [ ]:
# Volta escapa original para Plotar
trainPredict = scaler.inverse_transform(trainPredict)
TreinoY = scaler.inverse_transform([TreinoY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# Criando vetores de train e teste preditos para plotar
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=df.data, y=df.casos, line={'color': 'royalblue'}, name='valores reais')
fig.add_scattergl(x=df.data, y=trainPredictPlot[:,0], line={'color': 'green'}, name='ajuste_treino')
fig.add_scattergl(x=df.data, y=testPredictPlot[:,0], line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
print('Resultados treino')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
trainScore = mean_absolute_error(TreinoY[0], trainPredict[:,0])
print('MAE: %.2f' % (trainScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
trainScore = math.sqrt(mean_squared_error(TreinoY[0], trainPredict[:,0]))
print('RMSE: %.2f' % (trainScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

print()

print('Resultados teste')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('MAE: %.2f' % (testScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('RMSE: %.2f' % (testScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

## LSTM: Obitos

In [ ]:
# Semente do aleatório fixada
numpy.random.seed(7)

In [ ]:
# Seta os dados
dataset = df['obitos'].values
dataset = dataset.astype('float32')

In [ ]:
# Normaliza para facilitar nos calculos,  entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))

In [ ]:
# Separação entre Treino e Teste
train_fraction = 0.70
train_size = int(len(dataset) * train_fraction)
test_size = len(dataset) - train_size
Treino, Teste = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# Reshape para X=t e Y=t+1
look_back = 20
TreinoX, TreinoY = create_dataset(Treino, look_back)
testX, testY = create_dataset(Teste, look_back)
TreinoX = numpy.reshape(TreinoX, (TreinoX.shape[0], TreinoX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(LSTM(units=16, input_shape=(look_back, 1), return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=64))
model.add(Dense(units=1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
start_time = datetime.now()
history = model.fit(TreinoX, TreinoY, epochs = 300, batch_size = 10)
final_time = datetime.now()

In [ ]:
aux = final_time - start_time
print(aux.total_seconds()/60)

In [ ]:
y_ajustado = model.predict(TreinoX)
aux1 = TreinoY
aux2 = y_ajustado[:,0]
y_predito = model.predict(testX)
aux3 = testY
aux4 = y_predito[:,0]

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=train_visu.data, y=aux1, line={'color': 'royalblue'}, name='treino')
fig.add_scattergl(x=train_visu.data, y=aux2, line={'color': 'orangered'}, name='ajuste_treino')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=test_visu.data, y=aux3, line={'color': 'royalblue'}, name='teste')
fig.add_scattergl(x=test_visu.data, y=aux4, line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
# Predizer os dados
trainPredict = model.predict(TreinoX)
testPredict = model.predict(testX)

In [ ]:
# Volta escapa original para Plotar
trainPredict = scaler.inverse_transform(trainPredict)
TreinoY = scaler.inverse_transform([TreinoY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# Criando vetores de train e teste preditos para plotar
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=df.data, y=df.obitos, line={'color': 'royalblue'}, name='valores reais')
fig.add_scattergl(x=df.data, y=trainPredictPlot[:,0], line={'color': 'green'}, name='ajuste_treino')
fig.add_scattergl(x=df.data, y=testPredictPlot[:,0], line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
print('Resultados treino')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
trainScore = mean_absolute_error(TreinoY[0], trainPredict[:,0])
print('MAE: %.2f' % (trainScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
trainScore = math.sqrt(mean_squared_error(TreinoY[0], trainPredict[:,0]))
print('RMSE: %.2f' % (trainScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

print()

print('Resultados teste')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('MAE: %.2f' % (testScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('RMSE: %.2f' % (testScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

## GRU: Casos

In [ ]:
# Semente do aleatório fixada
numpy.random.seed(7)

In [ ]:
# Seta os dados
dataset = df['casos'].values
dataset = dataset.astype('float32')

In [ ]:
# Normaliza para facilitar nos calculos,  entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))

In [ ]:
# Separação entre Treino e Teste
train_fraction = 0.70
train_size = int(len(dataset) * train_fraction)
test_size = len(dataset) - train_size
Treino, Teste = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# Reshape para X=t e Y=t+1
look_back = 20
TreinoX, TreinoY = create_dataset(Treino, look_back)
testX, testY = create_dataset(Teste, look_back)
TreinoX = numpy.reshape(TreinoX, (TreinoX.shape[0], TreinoX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(GRU(units=16, input_shape=(look_back, 1), return_sequences=True))
model.add(GRU(units=32, return_sequences=True))
model.add(GRU(units=64))
model.add(Dense(units=1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
start_time = datetime.now()
history = model.fit(TreinoX, TreinoY, epochs = 300, batch_size = 10)
final_time = datetime.now()

In [ ]:
aux = final_time - start_time
print(aux.total_seconds()/60)

In [ ]:
y_ajustado = model.predict(TreinoX)
aux1 = TreinoY
aux2 = y_ajustado[:,0]
y_predito = model.predict(testX)
aux3 = testY
aux4 = y_predito[:,0]

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=train_visu.data, y=aux1, line={'color': 'royalblue'}, name='treino')
fig.add_scattergl(x=train_visu.data, y=aux2, line={'color': 'orangered'}, name='ajuste_treino')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=test_visu.data, y=aux3, line={'color': 'royalblue'}, name='teste')
fig.add_scattergl(x=test_visu.data, y=aux4, line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
# Predizer os dados
trainPredict = model.predict(TreinoX)
testPredict = model.predict(testX)

In [ ]:
# Volta escapa original para Plotar
trainPredict = scaler.inverse_transform(trainPredict)
TreinoY = scaler.inverse_transform([TreinoY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# Criando vetores de train e teste preditos para plotar
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=df.data, y=df.casos, line={'color': 'royalblue'}, name='valores reais')
fig.add_scattergl(x=df.data, y=trainPredictPlot[:,0], line={'color': 'green'}, name='ajuste_treino')
fig.add_scattergl(x=df.data, y=testPredictPlot[:,0], line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
print('Resultados treino')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
trainScore = mean_absolute_error(TreinoY[0], trainPredict[:,0])
print('MAE: %.2f' % (trainScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
trainScore = math.sqrt(mean_squared_error(TreinoY[0], trainPredict[:,0]))
print('RMSE: %.2f' % (trainScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

print()

print('Resultados teste')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('MAE: %.2f' % (testScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('RMSE: %.2f' % (testScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

## GRU: Obitos

In [ ]:
# Semente do aleatório fixada
numpy.random.seed(7)

In [ ]:
# Seta os dados
dataset = df['obitos'].values
dataset = dataset.astype('float32')

In [ ]:
# Normaliza para facilitar nos calculos,  entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset.reshape(-1, 1))

In [ ]:
# Separação entre Treino e Teste
train_fraction = 0.70
train_size = int(len(dataset) * train_fraction)
test_size = len(dataset) - train_size
Treino, Teste = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

In [ ]:
# Reshape para X=t e Y=t+1
look_back = 20
TreinoX, TreinoY = create_dataset(Treino, look_back)
testX, testY = create_dataset(Teste, look_back)
TreinoX = numpy.reshape(TreinoX, (TreinoX.shape[0], TreinoX.shape[1], 1))
testX = numpy.reshape(testX, (testX.shape[0], testX.shape[1], 1))

In [ ]:
model = Sequential()
model.add(GRU(units=16, input_shape=(look_back, 1), return_sequences=True))
model.add(GRU(units=32, return_sequences=True))
model.add(GRU(units=64))
model.add(Dense(units=1))

model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

In [ ]:
start_time = datetime.now()
history = model.fit(TreinoX, TreinoY, epochs = 300, batch_size = 10)
final_time = datetime.now()

In [ ]:
aux = final_time - start_time
print(aux.total_seconds()/60)

In [ ]:
y_ajustado = model.predict(TreinoX)
aux1 = TreinoY
aux2 = y_ajustado[:,0]
y_predito = model.predict(testX)
aux3 = testY
aux4 = y_predito[:,0]

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=train_visu.data, y=aux1, line={'color': 'royalblue'}, name='treino')
fig.add_scattergl(x=train_visu.data, y=aux2, line={'color': 'orangered'}, name='ajuste_treino')
fig.show()

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=test_visu.data, y=aux3, line={'color': 'royalblue'}, name='teste')
fig.add_scattergl(x=test_visu.data, y=aux4, line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
# Predizer os dados
trainPredict = model.predict(TreinoX)
testPredict = model.predict(testX)

In [ ]:
# Volta escapa original para Plotar
trainPredict = scaler.inverse_transform(trainPredict)
TreinoY = scaler.inverse_transform([TreinoY])
testPredict = scaler.inverse_transform(testPredict)
testY = scaler.inverse_transform([testY])

In [ ]:
# Criando vetores de train e teste preditos para plotar
trainPredictPlot = numpy.empty_like(dataset)
trainPredictPlot[:, :] = numpy.nan
trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict

testPredictPlot = numpy.empty_like(dataset)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict

In [ ]:
fig = go.Figure()
fig.add_scattergl(x=df.data, y=df.obitos, line={'color': 'royalblue'}, name='valores reais')
fig.add_scattergl(x=df.data, y=trainPredictPlot[:,0], line={'color': 'green'}, name='ajuste_treino')
fig.add_scattergl(x=df.data, y=testPredictPlot[:,0], line={'color': 'orangered'}, name='previsão')
fig.show()

In [ ]:
print('Resultados treino')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
trainScore = mean_absolute_error(TreinoY[0], trainPredict[:,0])
print('MAE: %.2f' % (trainScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
trainScore = math.sqrt(mean_squared_error(TreinoY[0], trainPredict[:,0]))
print('RMSE: %.2f' % (trainScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))

print()

print('Resultados teste')
# Calculo do Erro Absoluto Médio (MAE) e apresenta o resultado
testScore = mean_absolute_error(testY[0], testPredict[:,0])
print('MAE: %.2f' % (testScore))

# Calculo da Raiz do Erro Médio Quadrado (RMSE) e apresenta o resultado
testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
print('RMSE: %.2f' % (testScore))

# Calculo do r2_score e apresenta o resultado
trainScore = r2_score(TreinoY[0], trainPredict[:,0])
print('r2_score: %.2f' % (trainScore))